<a href="https://colab.research.google.com/github/apr0vleptos/MultiDiProject/blob/master/report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSH Algorith with Cosine Document similarity

The part of the Locality-Sensitive-Hashing Algorithm consists of 3 parts,as we can see in the picture below:


*   Turn each documment in sets of k-shinglings
*   Through, the MinHashing Algorithm we turn the above sets into Signatures
*   testing



![lsh_diagramm](https://miro.medium.com/max/1904/1*27nQOTC79yfh5lzmL06Ieg.png)